# Update POS RFID settings

Module interaction:

- Environement selection
- Premise selection
- Device selection
- Update parameters:
- - Tag Population Estimate
- => update device rfid properties
- For POV antena
- Update parameters:
- - Tx Power in Dbm
- - Rx Sensitivity in Dbm
- - Rf Time per Ant
- => update pov rfid properties

## Select device to update

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import settingsUtils as settingsUtils

import json

In [ ]:
%%capture
status = True
myEnv = chooseEnv.chooseEnv();
if not myEnv.get('urlDP')
    text = 'No Environment selected '
    uiUtils.displayMessage('POS update',text)
    print(text)
    status = False

In [ ]:
if status:
    print("Organization: " + myEnv.get('urlDP'))

In [ ]:
%%capture
if status:
    myEnv = chooseEnv.chooseDevice(myEnv, deviceTypeSelector='POS')
    deviceId = myEnv['device']['id']
    if not deviceId:
        text = 'No Device selected '
        uiUtils.displayMessage('POS update',text)
        print(text)
        status = False
    else:
        print('Device Id: ' + deviceId)

## Update RFID settings

In [ ]:
%%capture
if status:
    result = deviceUtils.getDeviceById(myEnv, deviceId)
    if not result:
        text = 'POS updated : Can not retreive POS '+deviceId+'information'
        uiUtils.displayMessage('POS update',text)
        print(text)
        status = False        

In [ ]:
if status:
    text = json.dumps(result, indent=4, sort_keys=True)
    uiUtils.displayMessage('POS update',text)
    print(text)

In [ ]:
if status:
    myForm = uiUtils.inputWithForm('Do you want to update device settings ' + result['device']['id'],['choice Y(yes) or N(no)?'])
    choice = myForm.get('choice Y(yes) or N(no)?').upper() == 'Y'


In [ ]:
%%capture
if status:
    if choice:
        settings = {}
        for mode in result['deviceMode']:
            device = settingsUtils.getRfidSettingsByDeviceId(myEnv, result['device']['id'], mode['id'])
            myForm = uiUtils.inputWithForm('RFID Profile Update for mode ' + mode['id'],['tagPopulationEstimate(integer)'])
            tagPopulationEstimate = myForm.get('tagPopulationEstimate(integer)')
            settings[result['device']['id'] + '_' + mode['id']] = (result['device']['id'], mode['id'], 'tagPopulationEstimate', int(tagPopulationEstimate))

In [ ]:
if status:
    if choice:
        for s in settings.values():
            settingsUtils.updateDeviceRfidSetting(myEnv, *s)

In [ ]:
if status:
    for pov in result['devicePov']:
        povId = pov['id']
        myForm = uiUtils.inputWithForm('Do you want to update pov settings ' + pov['name'],['choice Y(yes) or N(no)?'])
        choice = myForm.get('choice Y(yes) or N(no)?').upper() == 'Y'
        if choice:
            for mode in result['deviceMode']:
                myForm = uiUtils.inputWithForm('POV RFID Profile Update for pov ' + pov['name'] +' and mode ' + mode['name'],['txPowerinDbm(integer)','rxSensitivityinDbm(integer)','rfTimePerAnt(integer)'])
                txPowerinDbm = myForm.get('txPowerinDbm(integer)')
                rxSensitivityinDbm = myForm.get('rxSensitivityinDbm(integer)')
                rfTimePerAnt = myForm.get('rfTimePerAnt(integer)')
                settingsUtils.updatePovRfidSetting(myEnv, povId, mode['id'], 'txPowerinDbm', int(txPowerinDbm))
                settingsUtils.updatePovRfidSetting(myEnv, povId, mode['id'], 'rxSensitivityinDbm', int(rxSensitivityinDbm))
                settingsUtils.updatePovRfidSetting(myEnv, povId, mode['id'], 'rfTimePerAnt', int(rfTimePerAnt))  

## Get RFID settings

In [ ]:
%%capture
if status:
    result = deviceUtils.getDeviceById(myEnv, deviceId)
    if not result:
        text = 'POS updated : Can not retreive POS '+deviceId+' information'
        uiUtils.displayMessage('POS update',text)
        print(text)
        status = False  

In [ ]:
if status:
    text = json.dumps(result, indent=4, sort_keys=True)
    uiUtils.displayMessage('POS creation',text)
    print(text)

In [ ]:
%%capture
if status:
    settings = {}
    for mode in result['deviceMode']:
            settings[result['device']['id'] + '_' + mode['id']] = settingsUtils.getRfidSettingsByDeviceId(myEnv, result['device']['id'], mode['id'])
    if not settings:
        text = 'POS updated : Can not retreive POS '+deviceId+' settings'
        uiUtils.displayMessage('POS update',text)
        print(text)
        status = False  

In [ ]:
if status:
    text = json.dumps(settings, indent=4, sort_keys=True)
    uiUtils.displayMessage('POS creation',text)
    print(text)

In [ ]:
%%capture
if status:
    settings = {}
    for mode in result['deviceMode']:
        for pov in result['devicePov']:
            povId = pov['id']
            settings[povId + '_' + mode['id']] = settingsUtils.getRfidSettingsByPovId(myEnv, povId, mode['id'])
    if not settings:
        text = 'POS updated : Can not retreive POS PointOfView of device '+deviceId+' information'
        uiUtils.displayMessage('POS update',text)
        print(text)
        status = False 

In [ ]:
if status:
    text = json.dumps(settings, indent=4, sort_keys=True)
    uiUtils.displayMessage('POS creation',text)
    print(text)